<a href="https://colab.research.google.com/github/kovzanok/dls-final-task/blob/main/Final_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Установка зависимостей

In [3]:
!pip install insightface onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.5 MB/s eta 0:00:00


In [19]:
from torch import nn
from insightface.app import FaceAnalysis
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import torchvision.transforms.functional as TF
import cv2
from sklearn.preprocessing import LabelEncoder

import random
import os

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Подготовка данных

##Архив изображений

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!unzip -q /content/drive/MyDrive/img_align_celeba.zip -d /content/celeba/

In [8]:
def get_filenames_os(folder_path):
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    return files

In [9]:
filenames = get_filenames_os('/content/celeba/img_align_celeba')

In [10]:
idx = random.sample(range(len(filenames)), 100)

In [15]:
random_files = [ filenames[i] for i in idx ]

##Разметка "имя изображения  - id"

In [17]:
df = pd.read_csv('/content/drive/MyDrive/identity_CelebA.txt', delim_whitespace=True, header=None,index_col=0)

<ipython-input-17-2280069971>:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv('/content/drive/MyDrive/identity_CelebA.txt', delim_whitespace=True, header=None,index_col=0)


In [18]:
df.index.name = 'image_name'
df.rename(columns={1:'id'},inplace=True)

In [20]:
filtered_df = df.loc[random_files]
le = LabelEncoder()
filtered_df['encoded_id'] = le.fit_transform(filtered_df['id'])

#Пайплайн